In [52]:
import pandas as pd

data = pd.read_csv("features.csv")
data = data[["url", "label"]]
data.head()

,url,label
0,http://4aoo-bmmanager045288.vercel.app/?naps,0
1,http://0suz-bmmanager047181.vercel.app/?naps,0
2,http://ncrm-casefb588197.vercel.app/?naps,0
3,http://casefb668303-f2w6.vercel.app/?naps,0
4,http://casefb480777-qhn9.vercel.app/?naps,0


In [53]:
import tensorflow as tf
import numpy as np

# (A) Reproduzierbarkeit (optional)
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

### CNN

In [54]:
from sklearn.model_selection import train_test_split

# (B) Train/Val/Test-Split
X_train_df, X_test_df = train_test_split(data, test_size=0.2, stratify=data["label"], random_state=SEED)
X_train_df, X_val_df  = train_test_split(X_train_df, test_size=0.2, stratify=X_train_df["label"], random_state=SEED)

y_train = X_train_df["label"].astype("int32").to_numpy()
y_val   = X_val_df["label"].astype("int32").to_numpy()
y_test  = X_test_df["label"].astype("int32").to_numpy()

X_train_str = X_train_df["url"].astype(str).to_numpy()
X_val_str   = X_val_df["url"].astype(str).to_numpy()
X_test_str  = X_test_df["url"].astype(str).to_numpy()

In [105]:
# (C) Sequenzlänge auf Trainingsdaten festlegen (z. B. 95. Perzentil, min 8)
train_lengths = np.array([len(u) for u in X_train_str], dtype=np.int32)
MAX_LEN = max(8, int(np.percentile(train_lengths, 95)))

print(train_lengths)
print(MAX_LEN)

[33 52 94 ... 47 26 97]
107


In [56]:
 # (D) StringLookup-Vokabular nur auf TRAIN adaptieren:
#     - unicode_split: URLs -> Zeichen
#     - Ragged -> Batchweise
train_text_ds = tf.data.Dataset.from_tensor_slices(X_train_str).batch(1024)
char_ds = train_text_ds.map(lambda batch: tf.strings.unicode_split(batch, "UTF-8"))  # Ragged[str] pro Batch

In [57]:
from keras import layers

# StringLookup:
# - mask_token=""  -> Index 0 wird für Padding reserviert (Embedding mask_zero=True)
# - oov_token="[UNK]" -> Unbekannte Zeichen landen in OOV-Bucket
lookup = layers.StringLookup(
    mask_token="",
    oov_token="[UNK]",
    num_oov_indices=1,
    output_mode="int",
    name="char_lookup",
)
lookup.adapt(char_ds)  # Nur Trainingsdaten!
VOCAB_SIZE = lookup.vocabulary_size()  # inkl. PAD(0) + OOV
VOCAB_SIZE

619

In [58]:
# =======================================================
# MODELLDEFINITION (mit Vorverarbeitung im Graph)
# =======================================================
# Eingabe: roher URL-String
inp = layers.Input(shape=(), dtype=tf.string, name="input_url_str")

# 1) Unicode in Zeichen splitten (als Keras-Layer, damit kein KerasTensor-Fehler entsteht)
chars = layers.Lambda(lambda x: tf.strings.unicode_split(x, "UTF-8"),
                      name="unicode_split", output_shape=(None,))(inp)  # -> RaggedTensor[str]

# 2) Zeichen -> Indizes (PAD wird intern Index 0 sein, OOV > 0)
idx_rt = lookup(chars)  # lookup ist ein StringLookup-Layer, zuvor auf TRAIN adaptiert

# 3) Ragged[int] -> Dense[int] mit fester Länge und PAD=0
#    WICHTIG: default_value muss gleichen dtype wie idx_rt haben!
idx = layers.Lambda(
    lambda rt: rt.to_tensor(
        default_value=tf.cast(0, rt.dtype),              # sichert gleichen dtype
        shape=[tf.shape(rt)[0], MAX_LEN]                 # (B, MAX_LEN)
    ),
    name="pad_to_max_len",
    output_shape=(MAX_LEN,)  # pro Sample: feste Länge MAX_LEN
)(idx_rt)

In [59]:
# -----------------------------------------------------------
# Embedding (mask_zero=True -> Index 0 wird als PAD maskiert)
# -----------------------------------------------------------
# >>> Embedding <<<
x = layers.Embedding(
    input_dim=VOCAB_SIZE,   # wird aus lookup.vocabulary_size() übernommen
    output_dim=64,          # gewünschte Embedding-Dimension
    name="embedding"
)(idx)


In [60]:
# ---------
# Dropout
# ---------
x = layers.Dropout(rate=0.2, name="Dropout")(x)

In [61]:
# PAD-Positionen explizit auf 0 nullen (Feature-Vektoren dort = 0)
pad_mask = layers.Lambda(
    lambda t: tf.cast(tf.not_equal(t, 0), x.dtype),  # (B, T), 1 = echt, 0 = PAD
    name="pad_mask"
)(idx)
pad_mask = layers.Lambda(lambda m: tf.expand_dims(m, -1), name="pad_mask_expand")(pad_mask)  # (B, T, 1)
x = layers.Multiply(name="apply_pad_mask")([x, pad_mask])  # (B, T, 64), PAD exakt 0


# -------
# Conv1D
# -------
x = layers.Conv1D(
    filters=64,
    kernel_size=5,
    activation="relu",
    use_bias=False,
    name="Conv1D"
)(x)

In [62]:
# --------------
# MaxPooling1D
# --------------
x = layers.MaxPooling1D(pool_size=4, name="MaxPooling1D")(x)


In [63]:
# ----------------------
# GlobalMaxPooling1D
# ----------------------
x = layers.GlobalMaxPooling1D(name="GlobalMaxPooling1D")(x)


In [64]:
# -----
# Dense
# -----
# (zweiklassige Ausgabe, softmax)
out = layers.Dense(units=2, activation="softmax", name="Dense_Output")(x)


In [65]:
from keras import models

# -------------------------
# Modell zusammenbauen
# -------------------------
model = models.Model(inp, out, name="cnn_phishing_urls_charlevel")


In [66]:
# -------------------------
# Kompilieren
# -------------------------
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [67]:
from keras import callbacks

#-------------------------
# Early Stopping
# -------------------------
early_stop = callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

In [68]:
# -------------------------
# Training
# -------------------------
history = model.fit(
    X_train_str,
    y_train,
    validation_data=(X_val_str, y_val),
    epochs=30,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1,
)

Epoch 1/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6571 - loss: 0.5968 - val_accuracy: 0.8358 - val_loss: 0.3804
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8530 - loss: 0.3487 - val_accuracy: 0.8899 - val_loss: 0.2865
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8869 - loss: 0.2718 - val_accuracy: 0.9086 - val_loss: 0.2462
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9095 - loss: 0.2287 - val_accuracy: 0.9150 - val_loss: 0.2258
Epoch 5/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9222 - loss: 0.2030 - val_accuracy: 0.9182 - val_loss: 0.2140
Epoch 6/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9297 - loss: 0.1854 - val_accuracy: 0.9234 - val_loss: 0.2049
Epoch 7/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9389 - loss: 0.1670 - val_accuracy: 0.9269 - val_loss: 0.1976
Epoch 8/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9411 - loss: 0.1578 - val_accu

In [69]:
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------
# Testen / Evaluieren
# -------------------------
test_loss, test_acc = model.evaluate(X_test_str, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# Klassifikationsbericht & Konfusionsmatrix
y_proba = model.predict(X_test_str, batch_size=512, verbose=0)
y_pred  = np.argmax(y_proba, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4, target_names=["Phishing(0)", "Legit(1)"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Loss: 0.1766 | Test Accuracy: 0.9338

Classification Report:
              precision    recall  f1-score   support

 Phishing(0)     0.9370    0.8804    0.9079      1589
    Legit(1)     0.9321    0.9652    0.9484      2704

    accuracy                         0.9338      4293
   macro avg     0.9346    0.9228    0.9281      4293
weighted avg     0.9340    0.9338    0.9334      4293

Confusion Matrix:
[[1399  190]
 [  94 2610]]


In [70]:
import pandas as pd
CLASS_NAMES = ["Phishing (0)", "Legitim (1)"]

def predict_urls_batch(model, urls: list[str]) -> pd.DataFrame:
    x = np.array(urls, dtype=object)
    proba = model.predict(x, verbose=0)  # shape: (N, 2)
    pred_idx = proba.argmax(axis=1)
    df = pd.DataFrame({
        "url": urls,
        "pred_index": pred_idx,
        "pred_label": [CLASS_NAMES[i] for i in pred_idx],
        "proba_phishing": proba[:, 0],
        "proba_legitim":  proba[:, 1],
    })
    return df

# Beispiel:
urls = ["https://www.google.de/", "http://suspicious.example/offer?=free", "http://yield.beefyhubs.lol", "https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup"]
print(predict_urls_batch(model, urls))

                                                 url  pred_index  \
0                             https://www.google.de/           1   
1              http://suspicious.example/offer?=free           1   
2                         http://yield.beefyhubs.lol           0   
3  https://www.tensorflow.org/api_docs/python/tf/...           1   

     pred_label  proba_phishing  proba_legitim  
0   Legitim (1)        0.063284       0.936716  
1   Legitim (1)        0.441414       0.558586  
2  Phishing (0)        0.906884       0.093116  
3   Legitim (1)        0.001245       0.998755  


### LSTM

In [71]:
# >>> Embedding <<<
x = layers.Embedding(
    input_dim=VOCAB_SIZE,   # wird aus lookup.vocabulary_size() übernommen
    output_dim=64,          # gewünschte Embedding-Dimension
    name="embedding"
)(idx)

In [72]:
# Explizit: PAD-Vektoren auf 0 setzen (sicher stellt, dass PADDING nichts beiträgt)
pad_mask_float = layers.Lambda(lambda t: tf.cast(tf.not_equal(t, 0), x.dtype), name="pad_mask_float")(idx)  # (B, T)
x = layers.Multiply(name="apply_pad_mask")([x, layers.Lambda(lambda m: tf.expand_dims(m, -1))(pad_mask_float)])  # (B, T, 64)


In [73]:
# Dropout
x = layers.Dropout(0.2, name="dropout")(x)

In [74]:
# Conv1D ohne Bias (Nullfenster erzeugen keine Aktivierung)
x = layers.Conv1D(filters=64, kernel_size=5, activation="relu", use_bias=False, name="conv1d")(x)

In [75]:
# MaxPooling1D
x = layers.MaxPooling1D(pool_size=4, name="maxpool")(x)  # (B, T', 64)

In [76]:
# >>> Maske für LSTM passend zum gepoolten Zeitmaß berechnen <<<
# Ursprungsmaske (B, T, 1)
mask_expanded = layers.Lambda(lambda m: tf.expand_dims(m, -1), name="mask_expand")(pad_mask_float)  # (B, T, 1)
# Mit derselben Pooling-Config auf die Maske anwenden (ANY in Fenster -> True)
mask_pooled = layers.MaxPooling1D(pool_size=4, name="mask_pool")(mask_expanded)  # (B, T', 1)
mask_lstm = layers.Lambda(lambda m: tf.squeeze(tf.cast(m > 0, tf.bool), axis=-1), name="mask_bool")(mask_pooled)  # (B, T')

In [77]:
# >>> LSTM statt GlobalMaxPooling1D <<<
x = layers.LSTM(64, name="lstm")(x, mask=mask_lstm)  # (B, 64)

In [78]:
# Output
out = layers.Dense(2, activation="softmax", name="output")(x)

In [79]:
from keras import models

# -------------------------
# Modell zusammenbauen
# -------------------------
model = models.Model(inp, out, name="lstm_phishing_urls_charlevel")


In [80]:
# -------------------------
# Kompilieren
# -------------------------
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [81]:
from keras import callbacks

#-------------------------
# Early Stopping
# -------------------------
early_stop = callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

In [82]:
# -------------------------
# Training
# -------------------------
history = model.fit(
    X_train_str,
    y_train,
    validation_data=(X_val_str, y_val),
    epochs=30,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1,
)

Epoch 1/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6766 - loss: 0.5739 - val_accuracy: 0.8328 - val_loss: 0.3744
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8673 - loss: 0.2973 - val_accuracy: 0.8928 - val_loss: 0.2585
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9045 - loss: 0.2237 - val_accuracy: 0.9089 - val_loss: 0.2255
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9195 - loss: 0.1867 - val_accuracy: 0.9115 - val_loss: 0.2310
Epoch 5/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9240 - loss: 0.1808 - val_accuracy: 0.9185 - val_loss: 0.2030
Epoch 6/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9353 - loss: 0.1579 - val_accuracy: 0.9249 - val_loss: 0.1945
Epoch 7/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9369 - loss: 0.1490 - val_accuracy: 0.9278 - val_loss: 0.2033
Epoch 8/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9398 - loss: 0.1448 - val_accu

In [83]:
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------
# Testen / Evaluieren
# -------------------------
test_loss, test_acc = model.evaluate(X_test_str, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# Klassifikationsbericht & Konfusionsmatrix
y_proba = model.predict(X_test_str, batch_size=512, verbose=0)
y_pred  = np.argmax(y_proba, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4, target_names=["Phishing(0)", "Legit(1)"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Loss: 0.1663 | Test Accuracy: 0.9359

Classification Report:
              precision    recall  f1-score   support

 Phishing(0)     0.9334    0.8905    0.9114      1589
    Legit(1)     0.9373    0.9626    0.9498      2704

    accuracy                         0.9359      4293
   macro avg     0.9354    0.9266    0.9306      4293
weighted avg     0.9359    0.9359    0.9356      4293

Confusion Matrix:
[[1415  174]
 [ 101 2603]]


In [84]:
import pandas as pd
CLASS_NAMES = ["Phishing (0)", "Legitim (1)"]

def predict_urls_batch(model, urls: list[str]) -> pd.DataFrame:
    x = np.array(urls, dtype=object)
    proba = model.predict(x, verbose=0)  # shape: (N, 2)
    pred_idx = proba.argmax(axis=1)
    df = pd.DataFrame({
        "url": urls,
        "pred_index": pred_idx,
        "pred_label": [CLASS_NAMES[i] for i in pred_idx],
        "proba_phishing": proba[:, 0],
        "proba_legitim":  proba[:, 1],
    })
    return df

# Beispiel:
urls = ["https://www.google.de/", "http://suspicious.example/offer?=free", "http://yield.beefyhubs.lol", "https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup"]
print(predict_urls_batch(model, urls))

                                                 url  pred_index  \
0                             https://www.google.de/           1   
1              http://suspicious.example/offer?=free           1   
2                         http://yield.beefyhubs.lol           0   
3  https://www.tensorflow.org/api_docs/python/tf/...           1   

     pred_label  proba_phishing  proba_legitim  
0   Legitim (1)        0.064909       0.935091  
1   Legitim (1)        0.277159       0.722841  
2  Phishing (0)        0.915081       0.084919  
3   Legitim (1)        0.001309       0.998691  


### BILSTM/CNN

In [98]:
import tensorflow as tf
from keras import layers, models

# Eingabe: roher URL-String
inp = layers.Input(shape=(), dtype=tf.string, name="input_url_str")

# Unicode -> Zeichen (Ragged[str]); output_shape nötig, damit Keras die Form kennt
chars = layers.Lambda(
    lambda x: tf.strings.unicode_split(x, "UTF-8"),
    name="unicode_split",
    output_shape=(None,)
)(inp)

# Ragged[str] -> Dense[str] (Padding mit "" für mask_token="")
padded_chars = layers.Lambda(
    lambda rt: rt.to_tensor(default_value="", shape=[tf.shape(rt)[0], MAX_LEN]),
    name="pad_to_max_len_str",
    output_shape=(MAX_LEN,)
)(chars)

# StringLookup -> Indizes (PAD=0 dank mask_token="")
idx = lookup(padded_chars)  # Tensor[int32] Form: (B, MAX_LEN)


In [99]:
# Embedding mit Maskierung (LSTM versteht Masken)
x = layers.Embedding(
    input_dim=lookup.vocabulary_size(),
    output_dim=64,
    mask_zero=True,
    name="embedding"
)(idx)

# Bidirektionale LSTM mit Sequenzausgabe
x = layers.Bidirectional(
    layers.LSTM(64, return_sequences=True),
    name="bilstm"
)(x)

# Sicherstellen, dass PAD-Zeitschritte wirklich 0 sind (optional, aber sauber für Conv)
pad_bool = layers.Lambda(lambda t: tf.not_equal(t, 0), name="pad_bool")(idx)      # (B, T)
pad_float = layers.Lambda(lambda b: tf.cast(b, x.dtype), name="pad_float")(pad_bool)
pad_float = layers.Lambda(lambda m: tf.expand_dims(m, -1), name="pad_expand")(pad_float)  # (B, T, 1)
x = layers.Multiply(name="apply_pad_after_lstm")([x, pad_float])

# CNN-Block (lokale Muster auf kontextualisierten Hiddens) – ohne Bias, damit 0-Fenster neutral sind
x = layers.Dropout(0.2, name="dropout")(x)
x = layers.Conv1D(64, 5, activation="relu", use_bias=False, name="conv1d")(x)
x = layers.MaxPooling1D(pool_size=4, name="maxpool")(x)

# Zusammenfassen der Zeitachse
x = layers.GlobalMaxPooling1D(name="global_maxpool")(x)

# Output
out = layers.Dense(2, activation="softmax", name="output")(x)

model_bilstm_cnn = models.Model(inp, out, name="hybrid_bilstm_cnn")


In [100]:
# -------------------------
# Kompilieren
# -------------------------
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [101]:
from keras import callbacks

#-------------------------
# Early Stopping
# -------------------------
early_stop = callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

In [102]:
# -------------------------
# Training
# -------------------------
history = model.fit(
    X_train_str,
    y_train,
    validation_data=(X_val_str, y_val),
    epochs=30,
    batch_size=128,
    callbacks=[early_stop],
    verbose=1,
)

Epoch 1/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9586 - loss: 0.1063 - val_accuracy: 0.9336 - val_loss: 0.1643
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9725 - loss: 0.0736 - val_accuracy: 0.9298 - val_loss: 0.1744
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9755 - loss: 0.0666 - val_accuracy: 0.9324 - val_loss: 0.1699
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9743 - loss: 0.0681 - val_accuracy: 0.9359 - val_loss: 0.1764
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


In [103]:
from sklearn.metrics import classification_report, confusion_matrix

# -------------------------
# Testen / Evaluieren
# -------------------------
test_loss, test_acc = model.evaluate(X_test_str, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

# Klassifikationsbericht & Konfusionsmatrix
y_proba = model.predict(X_test_str, batch_size=512, verbose=0)
y_pred  = np.argmax(y_proba, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4, target_names=["Phishing(0)", "Legit(1)"]))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Loss: 0.1762 | Test Accuracy: 0.9352

Classification Report:
              precision    recall  f1-score   support

 Phishing(0)     0.9426    0.8785    0.9094      1589
    Legit(1)     0.9314    0.9686    0.9496      2704

    accuracy                         0.9352      4293
   macro avg     0.9370    0.9236    0.9295      4293
weighted avg     0.9355    0.9352    0.9347      4293

Confusion Matrix:
[[1396  193]
 [  85 2619]]


In [104]:
import pandas as pd
CLASS_NAMES = ["Phishing (0)", "Legitim (1)"]

def predict_urls_batch(model, urls: list[str]) -> pd.DataFrame:
    x = np.array(urls, dtype=object)
    proba = model.predict(x, verbose=0)  # shape: (N, 2)
    pred_idx = proba.argmax(axis=1)
    df = pd.DataFrame({
        "url": urls,
        "pred_index": pred_idx,
        "pred_label": [CLASS_NAMES[i] for i in pred_idx],
        "proba_phishing": proba[:, 0],
        "proba_legitim":  proba[:, 1],
    })
    return df

# Beispiel:
urls = ["https://www.google.de/", "http://suspicious.example/offer?=free", "http://yield.beefyhubs.lol", "https://www.tensorflow.org/api_docs/python/tf/keras/layers/StringLookup"]
print(predict_urls_batch(model, urls))

                                                 url  pred_index  \
0                             https://www.google.de/           1   
1              http://suspicious.example/offer?=free           1   
2                         http://yield.beefyhubs.lol           0   
3  https://www.tensorflow.org/api_docs/python/tf/...           1   

     pred_label  proba_phishing  proba_legitim  
0   Legitim (1)        0.014444       0.985556  
1   Legitim (1)        0.251571       0.748429  
2  Phishing (0)        0.769697       0.230303  
3   Legitim (1)        0.000565       0.999435  
